 # The directory where the data csv file is located

In [1]:
cd C:\Users\LIMDAESUN\Desktop\카이스트관련\UIUC\Risk_Management\Project\FRM_preprocessing

C:\Users\LIMDAESUN\Desktop\카이스트관련\UIUC\Risk_Management\Project\FRM_preprocessing


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
DJX = pd.read_csv('DJI.csv',index_col=0)[['Close']]/100
DJX.index = pd.to_datetime(DJX.index)
Stock = pd.read_csv('stock.csv',parse_dates=['date']).set_index('date')[['TICKER','PRC','DIVAMT']]
Option = pd.read_csv('Option_7_10.csv',parse_dates = ['date','exdate']).set_index('date')
Option = Option[list(Option.columns[:10]) + [Option.columns[11]]]
Div_date = Stock[pd.isna(Stock.DIVAMT) == False]
Stock = Stock.drop('DIVAMT', axis = 1)

In [4]:
DJX_Option = Option[Option.ticker == 'DJX'].dropna()
DJX_C = DJX_Option[DJX_Option.cp_flag == 'C']
DJX_P = DJX_Option[DJX_Option.cp_flag == 'P']

In [5]:
Stock_TICKER_option = sorted(list(set(Option.ticker) - {'DJX'} - {'DOW'}))
Stock_TICKER_stock = sorted(list(set(Stock.TICKER)))
my_ticker = sorted(list(set(Stock_TICKER_option)&set(Stock_TICKER_stock)))

In [6]:
Call_Option = Option[Option.cp_flag == 'C']
Put_Option = Option[Option.cp_flag == 'P']

In [7]:
Call = []
Put = []
my_Stock = pd.DataFrame([])
for i in my_ticker :
    Call.append(Call_Option[Call_Option.ticker == i])
    Put.append(Put_Option[Put_Option.ticker == i])
    my_Stock = pd.concat([my_Stock,Stock[Stock.TICKER == i].dropna().rename(columns = {'PRC' : i})[[i]]], axis = 1)

In [9]:
ATM_Call_lst = []
for n , i in enumerate(my_Stock.columns) :
    ticker_name = i # Ticker
    temp_df = pd.DataFrame([])
    day_index = sorted(list(set(Call[n].index))) # Ticker's traded day
    for j in day_index :
        strike_price = Call[n].loc[j].strike_price
        today_stock_price = my_Stock[ticker_name].loc[j]
        atm_index = np.abs(np.array(strike_price - today_stock_price*1000)).argmin()
        temp_df = pd.concat([temp_df,Call[n].loc[[j]].iloc[[atm_index]]], axis = 0)
    ATM_Call_lst.append(temp_df)
    
ATM_Put_lst = []
for n , i in enumerate(my_Stock.columns) :
    ticker_name = i # Ticker
    temp_df = pd.DataFrame([])
    day_index = sorted(list(set(Put[n].index))) # Ticker's traded day
    for j in day_index :
        strike_price = Put[n].loc[j].strike_price
        today_stock_price = my_Stock[ticker_name].loc[j]
        atm_index = np.abs(np.array(strike_price - today_stock_price*1000)).argmin()
        temp_df = pd.concat([temp_df,Put[n].loc[[j]].iloc[[atm_index]]], axis = 0)
    ATM_Put_lst.append(temp_df)

In [10]:
C_data = pd.DataFrame([])
for i in range(len(ATM_Call_lst)) :
    C_data = pd.concat([C_data,ATM_Call_lst[i]], axis = 1)
P_data = pd.DataFrame([])
for i in range(len(ATM_Put_lst)) :
    P_data = pd.concat([P_data,ATM_Put_lst[i]], axis = 1)
    
big_col = list(np.array((list(my_Stock.columns) * len(Option.columns))).reshape(len(Option.columns),-1).T.reshape(-1))
C_data.columns = [big_col, C_data.columns]
P_data.columns = [big_col, P_data.columns]

In [11]:
temp_df = pd.DataFrame([])
day_index = sorted(list(set(DJX_C.index)))
for i in day_index :
    strike_price = DJX_C.loc[i].strike_price
    today_index_price = DJX.Close.loc[i]
    atm_index = np.abs(np.array(strike_price - today_index_price*1000)).argmin()
    temp_df = pd.concat([temp_df, DJX_C.loc[[i]].iloc[[atm_index]]], axis = 0)
DJX_C_ATM = temp_df 

temp_df = pd.DataFrame([])
day_index = sorted(list(set(DJX_P.index)))
for i in day_index :
    strike_price = DJX_P.loc[i].strike_price
    today_index_price = DJX.Close.loc[i]
    atm_index = np.abs(np.array(strike_price - today_index_price*1000)).argmin()
    temp_df = pd.concat([temp_df, DJX_P.loc[[i]].iloc[[atm_index]]], axis = 0)

DJX_P_ATM = temp_df

In [12]:
my_Stock.to_excel('Stock_price.xlsx')
DJX.to_excel('Dow_jones_price.xlsx')
C_data.to_excel('C_ATM_7_10.xlsx')
P_data.to_excel('P_ATM_7_10.xlsx')
DJX_C_ATM.to_excel('DJX_C_7_10.xlsx')
DJX_P_ATM.to_excel('DJX_P_7_10.xlsx')
Div_date.to_excel('Stock_Div_date.xlsx')

In [13]:
def option_preprocessing(Option) :
    DJX_Option = Option[Option.ticker == 'DJX'].dropna()
    DJX_C = DJX_Option[DJX_Option.cp_flag == 'C']
    DJX_P = DJX_Option[DJX_Option.cp_flag == 'P']
    
    Stock_TICKER_option = sorted(list(set(Option.ticker) - {'DJX'} - {'DOW'}))
    Stock_TICKER_stock = sorted(list(set(Stock.TICKER)))
    my_ticker = sorted(list(set(Stock_TICKER_option)&set(Stock_TICKER_stock)))  
    
    Call_Option = Option[Option.cp_flag == 'C']
    Put_Option = Option[Option.cp_flag == 'P']
    
    Call = []
    Put = []
    my_Stock = pd.DataFrame([])
    for i in my_ticker :
        Call.append(Call_Option[Call_Option.ticker == i])
        Put.append(Put_Option[Put_Option.ticker == i])
        my_Stock = pd.concat([my_Stock,Stock[Stock.TICKER == i].dropna().rename(columns = {'PRC' : i})[[i]]], axis = 1)
        
    ATM_Call_lst = []
    for n , i in enumerate(my_Stock.columns) :
        ticker_name = i # Ticker
        temp_df = pd.DataFrame([])
        day_index = sorted(list(set(Call[n].index))) # Ticker's traded day
        for j in day_index :
            strike_price = Call[n].loc[j].strike_price
            today_stock_price = my_Stock[ticker_name].loc[j]
            atm_index = np.abs(np.array(strike_price - today_stock_price*1000)).argmin()
            temp_df = pd.concat([temp_df,Call[n].loc[[j]].iloc[[atm_index]]], axis = 0)
        ATM_Call_lst.append(temp_df)

    ATM_Put_lst = []
    for n , i in enumerate(my_Stock.columns) :
        ticker_name = i # Ticker
        temp_df = pd.DataFrame([])
        day_index = sorted(list(set(Put[n].index))) # Ticker's traded day
        for j in day_index :
            strike_price = Put[n].loc[j].strike_price
            today_stock_price = my_Stock[ticker_name].loc[j]
            atm_index = np.abs(np.array(strike_price - today_stock_price*1000)).argmin()
            temp_df = pd.concat([temp_df,Put[n].loc[[j]].iloc[[atm_index]]], axis = 0)
        ATM_Put_lst.append(temp_df)       
        
    C_data = pd.DataFrame([])
    for i in range(len(ATM_Call_lst)) :
        C_data = pd.concat([C_data,ATM_Call_lst[i]], axis = 1)
    P_data = pd.DataFrame([])
    for i in range(len(ATM_Put_lst)) :
        P_data = pd.concat([P_data,ATM_Put_lst[i]], axis = 1)

    big_col = list(np.array((list(my_Stock.columns) * len(Option.columns))).reshape(len(Option.columns),-1).T.reshape(-1))
    C_data.columns = [big_col, C_data.columns]
    P_data.columns = [big_col, P_data.columns]        
    
    temp_df = pd.DataFrame([])
    day_index = sorted(list(set(DJX_C.index)))
    for i in day_index :
        strike_price = DJX_C.loc[i].strike_price
        today_index_price = DJX.Close.loc[i]
        atm_index = np.abs(np.array(strike_price - today_index_price*1000)).argmin()
        temp_df = pd.concat([temp_df, DJX_C.loc[[i]].iloc[[atm_index]]], axis = 0)
    DJX_C_ATM = temp_df 

    temp_df = pd.DataFrame([])
    day_index = sorted(list(set(DJX_P.index)))
    for i in day_index :
        strike_price = DJX_P.loc[i].strike_price
        today_index_price = DJX.Close.loc[i]
        atm_index = np.abs(np.array(strike_price - today_index_price*1000)).argmin()
        temp_df = pd.concat([temp_df, DJX_P.loc[[i]].iloc[[atm_index]]], axis = 0)

    DJX_P_ATM = temp_df    
    
    return C_data , P_data , DJX_C_ATM , DJX_P_ATM

In [ ]:
Option = pd.read_csv('Option_11_15.csv',parse_dates = ['date','exdate']).set_index('date')
Option = Option[list(Option.columns[:10]) + [Option.columns[11]]]
C_data , P_data , DJX_C_ATM , DJX_P_ATM = option_preprocessing(Option)
C_data.to_excel('C_ATM_11_15.xlsx')
P_data.to_excel('P_ATM_11_15.xlsx')
DJX_C_ATM.to_excel('DJX_C_11_15.xlsx')
DJX_P_ATM.to_excel('DJX_P_11_15.xlsx')

In [ ]:
Option = pd.read_csv('Option_16_20.csv',parse_dates = ['date','exdate']).set_index('date')
Option = Option[list(Option.columns[:10]) + [Option.columns[11]]]
C_data , P_data , DJX_C_ATM , DJX_P_ATM = option_preprocessing(Option)
C_data.to_excel('C_ATM_16_20.xlsx')
P_data.to_excel('P_ATM_16_20.xlsx')
DJX_C_ATM.to_excel('DJX_C_16_20.xlsx')
DJX_P_ATM.to_excel('DJX_P_16_20.xlsx')

In [ ]:
Option = pd.read_csv('Option_21_25.csv',parse_dates = ['date','exdate']).set_index('date')
Option = Option[list(Option.columns[:10]) + [Option.columns[11]]]
C_data , P_data , DJX_C_ATM , DJX_P_ATM = option_preprocessing(Option)
C_data.to_excel('C_ATM_21_25.xlsx')
P_data.to_excel('P_ATM_21_25.xlsx')
DJX_C_ATM.to_excel('DJX_C_21_25.xlsx')
DJX_P_ATM.to_excel('DJX_P_21_25.xlsx')

In [ ]:
Option = pd.read_csv('Option_26_30.csv',parse_dates = ['date','exdate']).set_index('date')
Option = Option[list(Option.columns[:10]) + [Option.columns[11]]]
C_data , P_data , DJX_C_ATM , DJX_P_ATM = option_preprocessing(Option)
C_data.to_excel('C_ATM_26_30.xlsx')
P_data.to_excel('P_ATM_26_30.xlsx')
DJX_C_ATM.to_excel('DJX_C_26_30.xlsx')
DJX_P_ATM.to_excel('DJX_P_26_30.xlsx')

In [27]:
Option = pd.read_csv('Option_31_35.csv',parse_dates = ['date','exdate']).set_index('date')
Option = Option[list(Option.columns[:10]) + [Option.columns[11]]]
C_data , P_data , DJX_C_ATM , DJX_P_ATM = option_preprocessing(Option)
C_data.to_excel('C_ATM_31_35.xlsx')
P_data.to_excel('P_ATM_31_35.xlsx')
DJX_C_ATM.to_excel('DJX_C_31_35.xlsx')
DJX_P_ATM.to_excel('DJX_P_31_35.xlsx')